In [1]:
from train import *
from data.utils import *
DEVICE = torch.device("cpu")

/Users/lguerdan/.virtualenvs/10701-Project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trainset_cifar, testset_cifar = load_data(dataset='cifar')
trainloader_cifar = torch.utils.data.DataLoader(dataset=trainset_cifar, batch_size=32, shuffle=True)  
testloader_cifar  = torch.utils.data.DataLoader(dataset=testset_cifar,  batch_size=32, shuffle=False)

model = load_model(dataset='cifar')
train_results = train(
    model, trainloader_cifar, device=DEVICE, n_epochs=10)

test_results = test(
    model, testloader_cifar, device=DEVICE
)


Files already downloaded and verified
Files already downloaded and verified
Training 10 epoch(s) w/ 1563 batches each.


Testing...: 100%|██████████████████████████████████████████████| 313/313 [00:02<00:00, 106.63it/s]
